# ch.3　Python을 이용한 데이터 분석

## 3.7 추정

### 3.7.1 분석준비

In [16]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

%precision 3

%matplotlib inline

In [22]:
# 데이터 불러오기
fish = pd.read_csv("3-7-1-fish_length.csv")["length"]
fish

0    4.352982
1    3.735304
2    5.944617
3    3.798326
4    4.087688
5    5.265985
6    3.272614
7    3.526691
8    4.150083
9    3.736104
Name: length, dtype: float64

### 3.7.2 점추정

점추정: 모수를 어느 1개의 값으로 추정하는 추정방법  
- 표본평균을 추정값으로 모평균을 추정하듯이, 표본에서 평균값을 계산하는 것이다.
- 이때 표본평균을 사용할 수 있는건 표본평균이 불편성과 일치성을 가지고 있기 때문이다.  
~3.5절에서 겁나 열심히 알아본 내용이다.~

In [18]:
# 모평균의 점추정 파이썬코드로 구현
mu = np.mean(fish)
mu

4.187

In [19]:
# 표본의 불편분산
sigma_2 = np.var(fish, ddof = 1)
sigma_2

0.680

### 3.7.3 구간추정
구간추정: 추정값이 폭을 가지게 하는 추정 방법
- 이때 폭 계산을 위해 확률의 개념이 사용된다.
- 추정오차가 작으면, 구간추정의 폭이 좁아진다.
- 샘플사이즈가 커지면, 구간추정의 폭이 좁아진다.

### 3.7.4 신뢰계수와 신뢰구간
신뢰계수: 구간추정의 폭에 대한 신뢰정도를 확률로 표현한 값.
- 95%, 99% 가 자주 사용된다.  

신뢰구간: 특정 신뢰계수를 만족하는 구간

### 3.7.5 신뢰한계
신뢰한계: 신뢰구간의 하한값과 상한값.
  - 각각 하측신뢰한계, 상측신뢰한계라고 한다.

### 3.7.6 신뢰구간 계산 방법

구간추정시에는 t분포의 퍼센트포인트 개념을 이용한다. 
예를 들어 신뢰계수가 95%이고 t분포를 따른다면 2.5%와 97.5%의 퍼센트 포인트 사이의 구간에 t분포를 따르는 변수가 들어가는 것이다.

### 3.7.7 구간추정(실습)
구간추정에 필요한 정보: 자유도(샘플사이즈-1), 표본평균, 표준오차

In [20]:
# 자유도 구하기
df = len(fish) - 1
df

9

In [25]:
# 표준오차 구하기
sigma = np.std(fish, ddof=1)
se = sigma / np.sqrt(len(fish))
se

0.2608259396768776

In [26]:
# 신뢰구간 계산하기
interval = stats.t.interval(
    alpha = 0.95, df = df, loc = mu, scale = se)
interval

# alpha: 신뢰계수 / df: 자유도 / loc: 표본평균, scale: 표준오차

(3.597010056835825, 4.777068592173221)

### 3.7.8 신뢰구간을 구하는 방법 상세설명
1. 어떤 자유도를 가지는 t분포를 따르는 97.5% 지점을 계산한다.
    1. t분포를 따르는 97.5% 지점을 t_0.975라고표기한다.
    2. t분포는 평균에 대해 좌우대칭이기 때문에 2.5% 지점은 -t_0.975로 표기한다.
    3. t분포를 따르는 변수가 -t_0.975이상 t_0.975이하가 되는 확률이 95%이다.
        - 이때가 95% 신뢰계수가 된다.
2. 표본평균 - t_0.975 X 표준오차가 하측신뢰한계이다.
3. 표본평균 + t_0.975 X 표준오차가 상측신뢰한계이다.

In [9]:
# 97.5% 지점구하기
t_975 = stats.t.ppf(q = 0.975, df = df)
t_975

2.262

In [10]:
# 하측신뢰한계
lower = mu - t_975 * se
lower

3.597

In [11]:
# 상측신뢰한계
upper = mu + t_975 * se
upper

4.777

### 3.7.9 신뢰구간의 폭을 결정하는 요소

In [29]:
# 표본표준편차를 10배로 늘려서 95%신뢰구간 계산하기
se2 = (sigma*10) / np.sqrt(len(fish))
stats.t.interval(
    alpha = 0.95, df = df, loc = mu, scale = se2)
#신뢰구간이 넓어진다. 진짜 모평균을 찾기 힘들다는 말과 같다.

(-1.7132533521824618, 10.087332001191509)

In [31]:
# 샘플사이즈를 늘려보자.
df2 = (len(fish)*10) - 1
se3 = sigma / np.sqrt(len(fish)*10)
stats.t.interval(
    alpha = 0.95, df = df2, loc = mu, scale = se3)
#신뢰구간이 좁아진다. 평균값을 신뢰할 수 있다.

(4.0233803082774395, 4.350698340731607)

In [14]:
# 99%의 신뢰구간 구하기
stats.t.interval(
    alpha = 0.99, df = df, loc = mu, scale = se)
# 신뢰구간의 폭이 넓어진다.

(3.339, 5.035)

### 3.7.10 구간추정 결과의 해석

In [32]:
# 시행횟수 20000번 중 신뢰구간에 얼마나 해당되는지 알아보자.
be_included_array = np.zeros(20000, dtype = "bool")
be_included_array

array([False, False, False, ..., False, False, False])

In [34]:
# 95% 신뢰구간을 구하는 시행을 20000번 반복하여 배열을 채워준다.
#신뢰구간이 모평균 4를 포함하면 True
np.random.seed(1)
norm_dist = stats.norm(loc = 4, scale = 0.8)
for i in range(0, 20000):
    sample = norm_dist.rvs(size = 10)
    df = len(sample) - 1
    mu = np.mean(sample)
    std = np.std(sample, ddof = 1)
    se = std / np.sqrt(len(sample))
    interval = stats.t.interval(0.95, df, mu, se)
    if(interval[0] <= 4 and interval[1] >= 4):
        be_included_array[i] = True

In [35]:
#신뢰구간이 모퍙균 4 포함한 비율 구하기
sum(be_included_array) / len(be_included_array)

0.948

95에 가까운 값이 나온걸 볼 수 있다. 신기해라.